In [1]:
import numpy as np
import sys
sys.path.append('../')
import os
import pickle
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import multiprocessing as mp
import subprocess
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Layer, Conv2D, MaxPooling2D, Dense, Flatten, Input, Dropout, Conv2DTranspose, Reshape
from tensorflow.keras.backend import random_normal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy, mean_squared_error
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.regularizers import L1
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import tensorflow_probability as tfp

from models import *

In [2]:
# Load data
data = np.load('../FER_plus.npz', allow_pickle=True)

# Training data
X_train = np.expand_dims(data['xtrain'], axis=-1).astype(np.float) * 1./255
Y_train = data['ytrain']

print('Train: {}'.format(X_train.shape[0]))

# Validation data
X_val = np.expand_dims(data['xvalid'], axis=-1).astype(np.float) * 1./255
Y_val = data['yvalid']

print('Validation: {}'.format(X_val.shape[0]))

# Test data
X_test = np.expand_dims(data['xtest'], axis=-1).astype(np.float) * 1./255
Y_test = data['ytest']

print('Test: {}'.format(X_test.shape[0]))

labels = data['labels']

Train: 28709
Validation: 3589
Test: 3589


In [3]:
emotion_vae = EmotionVAE(100)
emotion_vae.summary()

Model: "EmotionVAE"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 48, 48, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 48, 48, 16)   160         encoder_input[0][0]              
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 24, 24, 16)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 24, 24, 32)   4640        max_pooling2d[0][0]              
_________________________________________________________________________________________

In [ ]:
emotion_vae.model.load_weights('./emotion_vae_ckpt')
#history = emotion_vae.fit(X_train, [X_train, Y_train], val_data=(X_val, [X_val, Y_val]), epochs=50, batch_size=32)